In [1]:
import cpa
import scanpy as sc
import pandas as pd
import numpy as np
import anndata as ad

[rank: 0] Global seed set to 0
2024-09-10 10:34:35,591	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-09-10 10:34:38,691	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
adata_train_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_train_4.h5ad"
adata_test_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_test_4.h5ad"
adata_ood_path = "/lustre/groups/ml01/workspace/ot_perturbation/data/combosciplex/adata_ood_4.h5ad"

In [3]:
adata_train = sc.read(adata_train_path)
adata_test = sc.read(adata_test_path)
adata_ood = sc.read(adata_ood_path)

In [4]:
try:
    data_path = 'combo_sciplex_prep_hvg_filtered.h5ad'
    adata = sc.read(data_path)
except:
    import gdown
    gdown.download('https://drive.google.com/uc?export=download&id=1RRV0_qYKGTvD3oCklKfoZQFYqKJy4l6t')
    data_path = 'combo_sciplex_prep_hvg_filtered.h5ad'
    adata = sc.read(data_path)

adata

adata.obs["index"] = adata.obs_names.values
adata.obs["index_adapted"] = adata.obs.apply(lambda x: x["index"][:-2], axis=1)
adata.obs.set_index("index_adapted", inplace=True)

adata_ood.obs["cov_drug_dose"] = adata.obs["cov_drug_dose"]
adata_ood.obs["condition_ID"] = adata.obs["condition_ID"]
adata_ood.obs["log_dose"] = adata.obs["log_dose"]
adata_ood.obs["smiles_rdkit"] = adata.obs["smiles_rdkit"]
adata_ood.obs["cell_type"] = adata.obs["cell_type"]

adata_test.obs["cov_drug_dose"] = adata.obs["cov_drug_dose"]
adata_test.obs["condition_ID"] = adata.obs["condition_ID"]
adata_test.obs["log_dose"] = adata.obs["log_dose"]
adata_test.obs["smiles_rdkit"] = adata.obs["smiles_rdkit"]
adata_test.obs["cell_type"] = adata.obs["cell_type"]

adata_train.obs["cov_drug_dose"] = adata.obs["cov_drug_dose"]
adata_train.obs["condition_ID"] = adata.obs["condition_ID"]
adata_train.obs["log_dose"] = adata.obs["log_dose"]
adata_train.obs["smiles_rdkit"] = adata.obs["smiles_rdkit"]
adata_train.obs["cell_type"] = adata.obs["cell_type"]

In [5]:
control_cells_gex = adata_ood[adata_ood.obs["condition"]=="control"].X.A
obs_names = list(adata_ood[adata_ood.obs["condition"]=="control"].obs_names)

all_data = []
obs_df = pd.DataFrame(columns=["condition_ID", "log_dose", "smiles_rdkit", "cell_type", "condition"])
indices = []

for condition in adata_ood.obs["condition"].cat.categories:
    if condition=="control":
        continue
    all_data.append(control_cells_gex)
    filtered_row = pd.Series(adata_ood[adata_ood.obs["condition"] == condition].obs[["condition_ID", "log_dose", "smiles_rdkit", "cell_type", "condition"]].iloc[0].values)
    filtered_row_df = pd.DataFrame([filtered_row.values], columns=["condition_ID", "log_dose", "smiles_rdkit", "cell_type", "condition"])
    duplicated_rows = pd.concat([filtered_row_df] * 500, ignore_index=True)
    obs_df = pd.concat([obs_df, duplicated_rows], ignore_index=True)

# Stack all data vertically to create a single array
all_data_array = np.vstack(all_data)


# Create the Anndata object
adata_ood_new = ad.AnnData(X=all_data_array, obs=obs_df)
adata_ood_new.var = adata_ood.var


control_cells_gex = adata_test[adata_test.obs["condition"]=="control"].X.A
obs_names = list(adata_test[adata_test.obs["condition"]=="control"].obs_names)

all_data = []
obs_df = pd.DataFrame(columns=["condition_ID", "log_dose", "smiles_rdkit", "cell_type", "condition"])
indices = []

for condition in adata_test.obs["condition"].cat.categories:
    if condition=="control":
        continue
    all_data.append(control_cells_gex)
    filtered_row = pd.Series(adata_test[adata_test.obs["condition"] == condition].obs[["condition_ID", "log_dose", "smiles_rdkit", "cell_type", "condition"]].iloc[0].values)
    filtered_row_df = pd.DataFrame([filtered_row.values], columns=["condition_ID", "log_dose", "smiles_rdkit", "cell_type", "condition"])
    duplicated_rows = pd.concat([filtered_row_df] * 500, ignore_index=True)
    obs_df = pd.concat([obs_df, duplicated_rows], ignore_index=True)

# Stack all data vertically to create a single array
all_data_array = np.vstack(all_data)


# Create the Anndata object
adata_test_new = ad.AnnData(X=all_data_array, obs=obs_df)
adata_test_new.var = adata_test.var

In [6]:
import anndata

adata_combined = anndata.concat((adata_train, adata_test_new, adata_ood_new), label="split", keys=["train", "test", "ood"], join="inner")

In [7]:
frac_valid = adata[adata.obs["split_1ct_MEC"] == "valid"].n_obs / (
    adata[adata.obs["split_1ct_MEC"] == "valid"].n_obs + adata[adata.obs["split_1ct_MEC"] == "train"].n_obs
)

In [8]:
import numpy as np


def create_split(x):
    if x["split"] != "train":
        return x["split"]
    is_train = np.random.choice(2, p=[frac_valid, 1 - frac_valid])
    if is_train:
        return "train_train"
    return "train_valid"


adata_combined.obs["split_1ct_MEC"] = adata_combined.obs.apply(create_split, axis=1)

In [9]:
cpa.CPA.setup_anndata(adata_combined,
                      perturbation_key='condition_ID',
                      dosage_key='log_dose',
                      control_group='CHEMBL504',
                      batch_key=None,
                      is_count_data=False,
                      categorical_covariate_keys=['cell_type'],
                      deg_uns_cat_key='cov_drug_dose',
                      max_comb_len=2,
                     )

100%|██████████| 61530/61530 [00:00<00:00, 1128347.00it/s]
An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


INFO     Generating sequential column names                                                                        
INFO     Generating sequential column names                                                                        


In [10]:
ae_hparams = {
    "n_latent": 128,
    "recon_loss": "gauss",
    "doser_type": "logsigm",
    "n_hidden_encoder": 512,
    "n_layers_encoder": 3,
    "n_hidden_decoder": 512,
    "n_layers_decoder": 3,
    "use_batch_norm_encoder": True,
    "use_layer_norm_encoder": False,
    "use_batch_norm_decoder": True,
    "use_layer_norm_decoder": False,
    "dropout_rate_encoder": 0.1,
    "dropout_rate_decoder": 0.1,
    "variational": False,
    "seed": 434,
}

trainer_params = {
    "n_epochs_kl_warmup": None,
    "n_epochs_pretrain_ae": 30,
    "n_epochs_adv_warmup": 50,
    "n_epochs_mixup_warmup": 3,
    "mixup_alpha": 0.1,
    "adv_steps": 2,
    "n_hidden_adv": 64,
    "n_layers_adv": 2,
    "use_batch_norm_adv": True,
    "use_layer_norm_adv": False,
    "dropout_rate_adv": 0.3,
    "reg_adv": 20.0,
    "pen_adv": 20.0,
    "lr": 0.0003,
    "wd": 4e-07,
    "adv_lr": 0.0003,
    "adv_wd": 4e-07,
    "adv_loss": "cce",
    "doser_lr": 0.0003,
    "doser_wd": 4e-07,
    "do_clip_grad": False,
    "gradient_clip_value": 1.0,
    "step_size_lr": 45,
}

In [11]:
model = cpa.CPA(
    adata=adata_combined,
    split_key="split_1ct_MEC",
    train_split="train_train",
    valid_split="train_valid",
    **ae_hparams,
)

[rank: 0] Global seed set to 434


In [12]:
model.train(
    max_epochs=2000,
    use_gpu=True,
    batch_size=128,
    plan_kwargs=trainer_params,
    early_stopping_patience=10,
    check_val_every_n_epoch=5,
    save_path="/lustre/groups/ml01/workspace/ot_perturbation/models/cpa/combosciplex/split_4",
)

100%|██████████| 32/32 [00:00<00:00, 207.72it/s]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA H100 80GB HBM3') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 5/2000:   0%|          | 4/2000 [01:00<7:49:44, 14.12s/it, v_num=1, recon=-2.5e+3, r2_mean=0.782, adv_loss=2.44, acc_pert=0.157]


Epoch 00004: cpa_metric reached. Module best state updated.


Epoch 10/2000:   0%|          | 9/2000 [02:06<7:11:57, 13.02s/it, v_num=1, recon=-2.97e+3, r2_mean=0.792, adv_loss=2.25, acc_pert=0.183, val_recon=-2.73e+3, disnt_basal=0.0693, disnt_after=0.0938, val_r2_mean=0.795, val_KL=nan]


Epoch 00009: cpa_metric reached. Module best state updated.



disnt_basal = 0.06740054575724867
disnt_after = 0.0882403511490468
val_r2_mean = 0.7993537041204948
val_r2_var = 0.43632849709499305
Epoch 15/2000:   1%|          | 14/2000 [03:11<7:05:10, 12.84s/it, v_num=1, recon=-3.16e+3, r2_mean=0.796, adv_loss=2.13, acc_pert=0.214, val_recon=-2.95e+3, disnt_basal=0.0674, disnt_after=0.0882, val_r2_mean=0.799, val_KL=nan]


Epoch 00014: cpa_metric reached. Module best state updated.


Epoch 20/2000:   1%|          | 19/2000 [04:17<7:06:58, 12.93s/it, v_num=1, recon=-3.32e+3, r2_mean=0.802, adv_loss=2.11, acc_pert=0.225, val_recon=-3.11e+3, disnt_basal=0.0653, disnt_after=0.083, val_r2_mean=0.804, val_KL=nan] 


Epoch 00019: cpa_metric reached. Module best state updated.



disnt_basal = 0.06135178456251478
disnt_after = 0.07774050360049524
val_r2_mean = 0.805595230225342
val_r2_var = 0.45480846251281853
Epoch 30/2000:   1%|▏         | 29/2000 [06:29<7:06:41, 12.99s/it, v_num=1, recon=-3.52e+3, r2_mean=0.801, adv_loss=2.11, acc_pert=0.234, val_recon=-3.29e+3, disnt_basal=0.0578, disnt_after=0.0727, val_r2_mean=0.807, val_KL=nan]
disnt_basal = 0.05566177694448728
disnt_after = 0.06883192453182044
val_r2_mean = 0.8065624070747351
val_r2_var = 0.45316126274104973
Epoch 40/2000:   2%|▏         | 39/2000 [08:32<6:32:39, 12.01s/it, v_num=1, recon=-3.62e+3, r2_mean=0.802, adv_loss=2.12, acc_pert=0.234, val_recon=-3.38e+3, disnt_basal=0.0549, disnt_after=0.0681, val_r2_mean=0.807, val_KL=nan]
disnt_basal = 0.054662515593995535
disnt_after = 0.06834998992823686
val_r2_mean = 0.8083666632625881
val_r2_var = 0.4498544917661607
Epoch 50/2000:   2%|▏         | 49/2000 [10:36<6:33:23, 12.10s/it, v_num=1, recon=-3.68e+3, r2_mean=0.801, adv_loss=2.2, acc_pert=0.213, val

In [13]:
model.predict(adata_combined)

100%|██████████| 1923/1923 [00:06<00:00, 306.90it/s]


In [14]:
adata_combined.write(
    "/lustre/groups/ml01/workspace/ot_perturbation/models/cpa/combosciplex/adata_with_predictions_4.h5ad"
)